In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import src.model as models
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

/Users/carlos/miniconda3/envs/TFG/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
VERSION = "4"
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV3"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [3]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [4]:
LEARNING_RATES = [1e-3, 3e-4, 1e-4, 3e-5, 1e-5]

In [5]:
OPTIMIZERS = {
    "Adam": lambda x: tf.keras.optimizers.legacy.Adam(learning_rate=x),
}

In [6]:
LOSS = {
    "BinaryFocalCrossentropy": lambda x, y: tf.keras.losses.BinaryFocalCrossentropy(
        alpha=x, gamma=y
    ),
}

ALPHA = 0.75

GAMMA = 2

In [7]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

2025-03-02 11:03:29.777628: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-03-02 11:03:29.777656: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-02 11:03:29.777661: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-02 11:03:29.777726: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-02 11:03:29.777924: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
BATCH_SIZE = 64

In [9]:
SUBVERSION = 0

for LEARNING_RATE in LEARNING_RATES:
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.MobileNetV3Large(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS["Adam"](LEARNING_RATE),
        loss=LOSS["BinaryFocalCrossentropy"](ALPHA, GAMMA),
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1

print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-03-02 11:03:33.523114: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - ETA: 0s - loss: 0.0781 - accuracy: 0.8759 - precision: 0.9123 - recall: 0.9204 - auc: 0.9346

2025-03-02 11:05:05.102252: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - 116s 403ms/step - loss: 0.0781 - accuracy: 0.8759 - precision: 0.9123 - recall: 0.9204 - auc: 0.9346 - val_loss: 0.0685 - val_accuracy: 0.9246 - val_precision: 0.9417 - val_recall: 0.9446 - val_auc: 0.9582
Epoch 2/10
280/280 [==============================] - 130s 466ms/step - loss: 0.0503 - accuracy: 0.9189 - precision: 0.9460 - recall: 0.9441 - auc: 0.9713 - val_loss: 0.0550 - val_accuracy: 0.9361 - val_precision: 0.9595 - val_recall: 0.9432 - val_auc: 0.9699
Epoch 3/10
280/280 [==============================] - 119s 425ms/step - loss: 0.0450 - accuracy: 0.9287 - precision: 0.9525 - recall: 0.9510 - auc: 0.9772 - val_loss: 0.0495 - val_accuracy: 0.9344 - val_precision: 0.9620 - val_recall: 0.9379 - val_auc: 0.9757
Epoch 4/10
280/280 [==============================] - 111s 398ms/step - loss: 0.0414 - accuracy: 0.9339 - precision: 0.9560 - recall: 0.9545 - auc: 0.9808 - val_loss: 0.0470 - val_accuracy: 0.9358 - val_precision: 0.9559 - val_reca

...

280/280 [==============================] - 110s 385ms/step - loss: 0.1770 - accuracy: 0.7451 - precision: 0.8646 - recall: 0.7683 - auc: 0.7931 - val_loss: 0.1699 - val_accuracy: 0.6685 - val_precision: 0.7255 - val_recall: 0.8025 - val_auc: 0.7486
Epoch 2/10
280/280 [==============================] - 100s 357ms/step - loss: 0.1399 - accuracy: 0.7726 - precision: 0.8509 - recall: 0.8393 - auc: 0.8056 - val_loss: 0.1496 - val_accuracy: 0.6983 - val_precision: 0.7334 - val_recall: 0.8547 - val_auc: 0.8007
Epoch 3/10
280/280 [==============================] - 106s 378ms/step - loss: 0.1257 - accuracy: 0.7972 - precision: 0.8579 - recall: 0.8695 - auc: 0.8366 - val_loss: 0.1375 - val_accuracy: 0.7147 - val_precision: 0.7424 - val_recall: 0.8709 - val_auc: 0.8300
Epoch 4/10
280/280 [==============================] - 108s 387ms/step - loss: 0.1170 - accuracy: 0.8101 - precision: 0.8654 - recall: 0.8797 - auc: 0.8570 - val_loss: 0.1291 - val_accuracy: 0.7231 - val_precision: 0.7487 - val_reca

INFO:tensorflow:Assets written to: /Users/carlos/WORKSPACE/MegaClassifier/models/MegaClassifier_a_MobileNetV3/v4/v4.4/assets


67/67 [==============================] - 22s 313ms/step - loss: 0.0989 - accuracy: 0.7674 - precision: 0.7782 - recall: 0.9069 - auc: 0.9084


2025-03-02 12:37:58.223589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


67/67 [==============================] - 21s 301ms/step



